# Twitter Pipeline v2

## Imports

In [1]:
path = '../'
import pandas as pd
import re
import sys
#import custom functions
sys.path.append('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/musow-pipeline/PYTHON_FILES')
from LogReg_Searches import LogRegSearches

## Variables

In [2]:
# descriptions training set -> v2 = musow+mji descriptions vs summarized scrapes from twitter searches  
archive_desc_training_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v2.pkl')

# twitter training set -> v1 = tweets from bigrams vs tweets for digital humanities and music company 
twitter_training_set_v1 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v1.pkl')

#kws to remove from url and title strings at various points 
discard = ['youtu', '404', 'Not Found', 'bandcamp', 'ebay', 'It needs a human touch', 'Page not found', 'open.spotify.com', 'We\'re sorry...', 'Not Acceptable!', 'Access denied', '412 Error', 'goo.gl', 'instagr.am', 'soundcloud', 'apple.co', 'amzn', 'masterstillmusic', 'Facebook', 'facebook', 'sheetmusiclibrary.website', 'Unsupported browser', 'Last.fm', 'last.fm', 'amazon.com', 'tidal.com', 'tmblr.co', 'blogspot', 'dailymusicroll', 'PortalTaxiMusic', 'apple.news', 'yahoo.com', 'sheetmusicplus.com', 'musicnotes.com', 'musescore.com', 'etsy', 'nts.live', 'twitch.tv', 'YouTube', 'radiosparx.com', 'freemusicarchive.org', 'blastradio', 'opensea', 'mixcloud', 'catalog.works', 'nft', 'NFT', 'allmusic.com', 'foundation.app', 'Robot or human?', 'heardle', 'insession.agency', 'jobvite', 'career', 'docs.google.com/forms/', 'discogs.com', 'zora.co', 'play.google.com', 't.me', 'mintable.app', 'instagram', 'linkedin', 'forms.gle', 'vimeo', 'radioiita', 'spotify', 'event']

## Training twitter and descriptions classifiers

This is a ONE TIME operation. The models are pickled and loaded later to predict new results from LOGREG_RELEVANCE/MODELS folder. 

In [ ]:
# one time training on twitter
#twitter_training_model = LogRegSearches.train(twitter_training_set_v1, 'tweet', 'Target', 10, 'precision', 'twitter', path)

# one time training on resources
#resource_training_model = LogRegSearches.train(archive_desc_training_v2, 'Description', 'Target', 10, 'f1','resources',path)

## Query Twitter

Calls Twitter API with a list of keywords and return results as raw csv and clean pickle in TWITTER_SEARCHES/RAW_SEARCHES folder.

In [3]:
#load token
token = 'AAAAAAAAAAAAAAAAAAAAAJgsNAEAAAAAQcsgbUnOJJmqmU483%2F8x6n9V1i8%3Df0qaEo9cV1sWP4eyNQ6E9s8BiRjvFTSN9mSqithe8uIXSNP68x'
#a selection of keywords based on MJI and musoW datasets
keywords = ['oral history', 'music magazine', 'sound archive', 'music history', 'music culture', 'music research', 'sheet music', 'music library', 'digital library', 'music collection', 'digital collection', 'sound recording', 'midi file', 'audio file', 'music information', 'musical score', 'digital score', 'song dataset', 'digital edition', 'digital archive', 'digital library', 'music archive', 'music library', 'archive collection']
#transfer keywords to input list w/ additional twitter parameters, e.g. no retweets
input_keywords = [f'\"{k}\" -is:retweet' for k in keywords] 
#input time periods for search as a comma separated list, searches will be conducted for each time pair e.g. first of week/month, last of week/month 
start = ['2022-05-01T00:00:00.000Z', '2022-05-02T00:00:00.000Z', '2022-05-03T00:00:00.000Z', '2022-05-04T00:00:00.000Z', '2022-05-05T00:00:00.000Z']
end = ['2022-05-01T23:59:59.000Z', '2022-05-02T23:59:59.000Z', '2022-05-03T23:59:59.000Z', '2022-05-04T23:59:59.000Z', '2022-05-05T23:59:59.000Z']
#input max results / counts, and path 
mresults = 500 # max tweets per json response (100-500)
mcount = 500 # max tweets per search period 
#run the search! 
for k in input_keywords:
    filename = re.sub(r"([^A-Za-z0-9]+)", '', k) + f'_{start[0][0:10]}' + f'_{end[-1][6:10]}'
    filename = re.sub(r"isretweet", '', filename)
    prediction_twitter = LogRegSearches.search_twitter(token, k, start, end, mresults, mcount, filename)

#today = date.today()
#week_ago = today - timedelta(days=7)
#start = [week_ago.strftime("%Y-%m-%dT%H:%M:%S.000Z")]
#end = [today.strftime("%Y-%m-%dT%H:%M:%S.000Z")]

-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-01T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added for '"oral history" -is:retweet': 100
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-02T00:00:00.000Z
# of Tweets added from this response:  142
Total # of Tweets added for '"oral history" -is:retweet': 242
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-03T00:00:00.000Z
# of Tweets added from this response:  135
Total # of Tweets added for '"oral history" -is:retweet': 377
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
-------------------
Start Date:  2022-05-04T00:00:00.000Z
# of Tweets added from this response:  149
Total # of Tweets added for '"oral history" -is:retweet': 526
-------------------
-------------------
Toke

## Classify tweets

In [3]:
#load all search results into a single dataframe 
tweets_to_classify = LogRegSearches.tweets_to_classify(path, '2022-05-01_5-05.pkl')

Total tweets to classify: 12333


In [4]:
#run classification and get links from results
predicted_tweets, twitter_link_list = LogRegSearches.predict_twitter(path, 'twitter', tweets_to_classify, 'tweet', 1, discard, '')

Total tweets classified: 1225


## Scrape URLS

In [ ]:
#scrape URL list
scraped_links = LogRegSearches.scrape_links(twitter_link_list)

#merge w/ predictions and save for reuse
twitter_scrapes_preds = pd.merge(predicted_tweets, scraped_links, on='URL')
twitter_scrapes_preds.to_pickle(path+'LOGREG_RELEVANCE/SCRAPES/all_keywords_2022-05-01_5-05_scrapes.pkl')

#check length
len(twitter_scrapes_preds)

## Classify web resources

In [6]:
predicted_resources = LogRegSearches.predict_resource(path, 'resources_v2', twitter_scrapes_preds, 'Description', 1, discard, 'all_keywords_2022-05-01_5-05')
predicted_resources

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,Title,Description
0,"""The Silent Film Sound and Music Archive (http...",1,4.390663,0.987759,229,http://www.sfsma.org,"""music archive"" -is:retweet",Silent Film Sound & Music Archive – a digital ...,"Since 2014, we've been digitizing and sharing..."
1,Listen to the most recent episode of my podcas...,1,3.697857,0.975822,169,https://anchor.fm/educa-web-radio/episodes/Mus...,"""music history"" -is:retweet",Music History com Fulvio Cristofoli - The Mote...,"Music History, desenvolvido e apresentado por..."
2,Check out this sheet music video I made of Laz...,1,3.498099,0.970634,127,https://ift.tt/pDMjZQh,"""sheet music"" -is:retweet",Reddit - Alla scoperta di ciò che ti piace,Check out this sheet music video I made of La...
3,BBF releases the May 2022 edition of FORUM VIE...,1,3.467769,0.969757,65,http://brokersforumofindia.com/magazines.php,"""digital edition"" -is:retweet",http://brokersforumofindia.com/magazines.php,An Award Winning - Monthly Capital Markets and...
4,"Happy birthday ! The Beaver, LSE Students' Uni...",1,3.453722,0.969342,293,https://digital.library.lse.ac.uk/browse#beaver,"""digital library"" -is:retweet",Browse | LSE Digital Library,"A collection of newspapers, journals, pamphle..."
...,...,...,...,...,...,...,...,...,...
255,Looking for troubles with Steve Erection Arous...,1,0.033814,0.508453,238,https://gruenholtz.com/product/fine-art-of-aro...,"""digital collection"" -is:retweet",Fine Art Of Arousal - Digital Collection - Gru...,The Fine Art of Erections is the very first p...
256,A fellow Brit wrote this in the Telegraph 30 ...,1,0.024075,0.506018,476,https://bit.ly/3rZrdqv,"""music history"" -is:retweet",History of music in the American Midwest,The American Midwest has often been sentiment...
257,Voyager by Hunter Gregory| J.W. Pepper Sheet M...,1,0.021628,0.505407,195,https://www.jwpepper.com/Voyager/11393487.item...,"""sheet music"" -is:retweet",Voyager by Hunter Gregory| J.W. Pepper Sheet M...,The piece uses carefully designed eighth-note...
258,Current status of ManiRatnam's PonniyinSelvan ...,1,0.017407,0.504352,187,https://m.behindwoods.com/tamil-movies-cinema-...,"""sound recording"" -is:retweet",Mani Ratnam and AR Rahman's Ponniyin Selvan Mo...,Director Mani Ratnam is directing the two-par...
